In [39]:
import graphviz
from graphviz import Digraph
import csv
import numpy as np
import util
from util import createPetri
from util import parsePlainFile
from util import remove_nodes
from util import add_nodes
from util import offsetEquation
from util import add_boxes
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import nltk
nltk.download('words')
from nltk.corpus import words
word_list = words.words()

[nltk_data] Downloading package words to /Users/shivam/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
if not os.path.isdir('descriptions'):
    os.mkdir('descriptions')

if not os.path.isdir('plain_data'):
    os.mkdir('plain_data')
    
if not os.path.isdir('edge_data'):
    os.mkdir('edge_data')
    
if not os.path.isdir('mask_data'):
    os.mkdir('mask_data')

In [15]:
maxNodes = 8
count = 0
NO_Samples = 100
for i in range(2, maxNodes):
    print (i)
    for j in range(NO_Samples):
        ranInts = np.random.randint(len(word_list), size=i)
        word_list = np.asarray(word_list)
        words = word_list[ranInts]
        createPetri(words, count)
        count += 1

2
3
4
5
6
7


In [20]:
import shutil

for i in os.listdir('data'):
    if ".png" not in i:
        os.remove(os.path.join('data', i))
        
for i in os.listdir('plain_data'):
    if ".plain" not in i:
        os.remove(os.path.join('plain_data', i))

In [45]:
for i in tqdm(os.listdir('plain_data'),position=0, leave=True):

#     print (i)
    file_id = i.split(".")[0]
    filePath = os.path.join('plain_data', i)
    offset_y, node_x_list, node_y_list, node_rx_list, node_ry_list, node_type_list = util.parsePlainFile(filePath)
    
    graphFile = os.path.join('data', str(file_id) + ".png")
    im = Image.open(graphFile)
    im = im.convert('1')
    edge_array = im
    for i in range(len(node_x_list)):
        edge_array = remove_nodes(offsetEquation(node_x_list[i]),\
                                 offsetEquation(offset_y-node_y_list[i]),\
                                 node_rx_list[i],\
                                 node_ry_list[i], np.array(edge_array), node_type=node_type_list[i] )
    edge_mask = 1 - np.uint8(edge_array)
    
    full_mask_elliptical = im
    for i in range(len(node_x_list)):
        if node_type_list[i] == "ellipse":
            full_mask_elliptical = add_nodes(offsetEquation(node_x_list[i]),\
                                     offsetEquation(offset_y-node_y_list[i]),\
                                     node_rx_list[i],\
                                     node_ry_list[i], np.array(full_mask_elliptical))
        else:
            full_mask_elliptical = remove_nodes(offsetEquation(node_x_list[i]),\
                                     offsetEquation(offset_y-node_y_list[i]),\
                                     node_rx_list[i],\
                                     node_ry_list[i], np.array(full_mask_elliptical),node_type=node_type_list[i])
    node_elliptical_mask = np.uint8(edge_array) - np.uint8(full_mask_elliptical)
    
    full_mask_box = im
    for i in range(len(node_x_list)):
        if node_type_list[i] == "box":
            full_mask_box = add_boxes(offsetEquation(node_x_list[i]),\
                                     offsetEquation(offset_y-node_y_list[i]),\
                                     node_rx_list[i],\
                                     node_ry_list[i], np.array(full_mask_box))
        else:
            full_mask_box = remove_nodes(offsetEquation(node_x_list[i]),\
                                     offsetEquation(offset_y-node_y_list[i]),\
                                     node_rx_list[i],\
                                     node_ry_list[i], np.array(full_mask_box),node_type=node_type_list[i])
    node_box_mask = np.uint8(edge_array) - np.uint8(full_mask_box)
    
    full_mask = im
    for i in range(len(node_x_list)):
        if node_type_list[i] == "ellipse":
            full_mask = add_nodes(offsetEquation(node_x_list[i]),\
                                     offsetEquation(offset_y-node_y_list[i]),\
                                     node_rx_list[i],\
                                     node_ry_list[i], np.array(full_mask))
        else:
            full_mask = add_boxes(offsetEquation(node_x_list[i]),\
                                     offsetEquation(offset_y-node_y_list[i]),\
                                     node_rx_list[i],\
                                     node_ry_list[i], np.array(full_mask))

#     new_image = Image.fromarray(np.uint8(node_box_mask)*255)
#     new_image.save("mask_data/" + str(file_id) + "_box.png")
    
    stack = np.stack((node_box_mask, node_elliptical_mask, full_mask, edge_mask), axis=-1)
    new_image = Image.fromarray(np.uint8(stack*255))
    new_image.save("mask_data/" + str(file_id) + ".png")
    
#     new_image = Image.fromarray(np.uint8(edge_mask)*255)
#     new_image.save("mask_data/" + str(file_id) + "_edge" + ".png")

#     new_image = Image.fromarray(np.uint8(node_elliptical_mask)*255)
#     new_image.save("mask_data/" + str(file_id) + "_ellipse" + ".png")
    
#     new_image = Image.fromarray(np.uint8((full_mask)*255))
    
#     new_image.save("mask_data/" + str(file_id) + "_background" + ".png")

100%|██████████| 600/600 [01:10<00:00,  8.54it/s]
